In [ ]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=7000)])
    except RuntimeError as e:
        print(e)

In [ ]:
from tensorflow.keras.applications import InceptionV3
from shutil import copyfile
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import os
from tensorflow.keras.preprocessing import image
import numpy as np
from PIL import ImageFile
import tensorflow as tf
from nipa.taskSubmit import nipa_submit


ImageFile.LOAD_TRUNCATED_IMAGES = True

%matplotlib inline
with tf.device('/device:GPU:0'):


    pre_trained_model = InceptionV3(input_shape=(244, 244, 3),
                                    weights="imagenet",
                                    include_top=False)


    for layer in pre_trained_model.layers:
        layer.trainable = False

#     pre_trained_model.summary()

    last_layer = pre_trained_model.get_layer("mixed9")
    print(f"The shape of last output layer is {last_layer.output_shape}")
    last_output = last_layer.output



    x = tf.keras.layers.Flatten()(last_output)

    x = tf.keras.layers.Dense(units=1024, activation="relu")(x)

    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Dense(1, activation="sigmoid")(x)


    model = Model(pre_trained_model.input, x)

    model.compile(loss="binary_crossentropy",
                  optimizer=Adam(learning_rate=1e-3),
                  metrics=["accuracy"])

    model.summary()


    train_datagen = ImageDataGenerator(rescale=1./255,
                                       rotation_range=45,
                                       width_shift_range=0.2,
                                       height_shift_range=0.2,
                                       shear_range=0.7,
                                       zoom_range=0.2,
                                       horizontal_flip=True,
                                       fill_mode="nearest")

    validation_datagen = ImageDataGenerator(rescale=1./255)


    train_generator = train_datagen.flow_from_directory(
        "/home/workspace/user-workspace/crack_data_full_8_2/train/",
        target_size=(244, 244),
        batch_size=30,
        class_mode="binary"
    )


    validation_generator = validation_datagen.flow_from_directory(
        "/home/workspace/user-workspace/crack_data_full_8_2/validation/",
        target_size=(244, 244),
        batch_size=32, 
        class_mode="binary"
    )


    history = model.fit(
        train_generator,
        steps_per_epoch=375, 
        epochs=10,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=88
    )

    
    fine_tune_at = 10

    pre_trained_model.trainable = True
    

    for layer in pre_trained_model.layers[-fine_tune_at:]:
        layer.trainable =  False

    

    
    model.compile(optimizer=Adam(learning_rate=2e-5),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

#   ## 재학습 진행
    history = model.fit(train_generator,
                        steps_per_epoch=375,
                        epochs=40,
                        validation_data=validation_generator,
                        validation_steps=88,
                        verbose=1)


acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(len(acc))

plt.plot(epochs, acc, "r", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and Validation accuracy")

plt.figure()

plt.plot(epochs, loss, "r", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and Validation loss")

plt.legend()
plt.show()
model.save('/home/workspace/user-workspace/HHD/model_save/InceptionV3_224_224.h5')